In [1]:
import pandas as pd
import numpy as np

In [2]:
import pandas as pd
import numpy as np
reviewsDF = pd.read_csv('DatasetFinal.csv')
reviewsDF['Brand Name'] = reviewsDF['Brand Name'].str.lower()
reviews = reviewsDF.Reviews.tolist()
brands = reviewsDF['Brand Name'].tolist()
ratings = reviewsDF['Rating'].tolist()
print("Loaded " + str(len(reviews)) + " reviews.")

Loaded 59255 reviews.


In [3]:
reviewsDF.head()

,Product Name,Brand Name,Rating,Reviews,Reviews_Votes
0,Samsung Electronics Galaxy Note 20 Ultra 5G Fa...,samsung,1,Received a used phone . Very disappointed!!,2
1,Samsung Electronics Galaxy Note 20 Ultra 5G Fa...,samsung,4,Update 8/31/20 IF YOU DIDN'T RECEIVE YOUR WIRE...,188
2,Samsung Electronics Galaxy Note 20 Ultra 5G Fa...,samsung,5,Great phone and food quality. I was able to us...,0
3,Samsung Electronics Galaxy Note 20 Ultra 5G Fa...,samsung,5,"La pantalla se ve impresionante, se escucha me...",30
4,Samsung Electronics Galaxy Note 20 Ultra 5G Fa...,samsung,1,"It came today. When opened the box, there was ...",1


In [4]:
import nltk
def get_all_phrases_containing_tar_wrd(target_word, tar_passage, left_margin = 5, right_margin = 5):
    """
        Function to get all the pharses that contain the target word in a text/passage tar_passage.
        Workaround to save the output given by nltk Concordance function
        
        str target_word --> aspect to be searched for
        str tar_passage  --> sentence extracted from a customer review
        int left_margin int right_margin --> left_margin and right_margin allocate the number of words/punctuation before and after target word
        Left margin will take note of the beginning of the text
    """
     
   
    tokens = nltk.word_tokenize(tar_passage)
    tokens = [x for x in tokens if len(x)>2] 
 
    text = nltk.Text(tokens)
 
 
    c = nltk.ConcordanceIndex(text.tokens, key = lambda s: s.lower())
 

    
    concordance_txt = ([text.tokens[  list(map(lambda x: x-5 if (x-left_margin)>0 else 0,[offset]))[0]:offset+right_margin] for offset in c.offsets(target_word)])

 
    return [''.join([x+' ' for x in con_sub]) for con_sub in concordance_txt]

In [5]:
def cleanText(raw_text, remove_stopwords=False, stemming=False, split_text=False):
    '''
    Convert a raw review to a cleaned review
    '''
    text = BeautifulSoup(raw_text, 'lxml').get_text()  #remove html
    letters_only = re.sub("[^a-zA-Z]", " ", text)  # remove non-character
    words = letters_only.lower().split() # convert to lower case 
    
    if remove_stopwords: # remove stopword
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
        
    if stemming==True: # stemming
#         stemmer = PorterStemmer()
        stemmer = SnowballStemmer('english') 
        words = [stemmer.stem(w) for w in words]
        
    if split_text==True:  # split text
        return (words)
    
    return( " ".join(words))

In [6]:
from nltk import sent_tokenize


relevant_aspects = ['battery', 'screen', 'camera', 'performance']


phrases = []
brand = []
rating = []
aspects = []
review_num = []
counter = 0

for review in reviews:
    try:
       
        for sentence in sent_tokenize(review):
            for  important_word in relevant_aspects:
               
                phrases_in_sentence = get_all_phrases_containing_tar_wrd(important_word, sentence, left_margin = 5, right_margin = 5) 
                for phrase in phrases_in_sentence:
                    brand.append(brands[counter])
                    rating.append(ratings[counter])
                    aspect = 'other'
                    for imp_word in relevant_aspects:
                        if imp_word in phrase.lower():
                            aspect = imp_word
                    aspects.append(aspect)
                    phrases.append(phrase)
                    review_num.append(counter)
    except:
        pass
    counter = counter + 1

print('Extracted ' + str(len(phrases)) + ' phrases from the reviews')


phrasesDF = pd.DataFrame()
phrasesDF['brand'] = brand 
phrasesDF['phrase'] = phrases
phrasesDF['rating'] = rating
phrasesDF['aspect'] = aspects
phrasesDF['review_id'] = review_num
phrasesDF.head()

Extracted 22131 phrases from the reviews


,brand,phrase,rating,aspect,review_id
0,samsung,even the note but this screen here bests all t...,4,screen,1
1,samsung,all this makes for outstanding screen,4,screen,1
2,samsung,Great main camera,4,camera,1
3,samsung,result but all all the camera top notch all its,4,camera,1
4,samsung,find phone with better main camera the market,4,camera,1


In [7]:
from textblob import TextBlob
from textblob import Blobber

In [8]:
def detect_polarity(reviewsDF):
    return TextBlob(reviewsDF).sentiment.polarity

In [9]:
df_tb = phrasesDF.phrase.apply(detect_polarity)

In [10]:
df_tb.head()

0    0.000000
1    0.500000
2    0.483333
3    0.500000
4    0.333333
Name: phrase, dtype: float64

In [11]:
nRow, nCol = phrasesDF.shape
print(f'There are {nRow} rows and {nCol} columns')

There are 22131 rows and 5 columns


In [12]:
phrasesDF["text_sentiment"] = phrasesDF.phrase.apply(detect_polarity)

In [13]:
phrasesDF.head()

,brand,phrase,rating,aspect,review_id,text_sentiment
0,samsung,even the note but this screen here bests all t...,4,screen,1,0.000000
1,samsung,all this makes for outstanding screen,4,screen,1,0.500000
2,samsung,Great main camera,4,camera,1,0.483333
3,samsung,result but all all the camera top notch all its,4,camera,1,0.500000
4,samsung,find phone with better main camera the market,4,camera,1,0.333333


In [14]:
def detect_subjectivity(reviewsDF):
    return TextBlob(reviewsDF).sentiment.subjectivity

In [15]:
phrasesDF["text_subjectivity"] = phrasesDF.phrase.apply(detect_subjectivity)

In [16]:
phrasesDF.head()

,brand,phrase,rating,aspect,review_id,text_sentiment,text_subjectivity
0,samsung,even the note but this screen here bests all t...,4,screen,1,0.000000,0.000000
1,samsung,all this makes for outstanding screen,4,screen,1,0.500000,0.875000
2,samsung,Great main camera,4,camera,1,0.483333,0.541667
3,samsung,result but all all the camera top notch all its,4,camera,1,0.500000,0.500000
4,samsung,find phone with better main camera the market,4,camera,1,0.333333,0.416667


In [17]:
phrasesDF['comp_score'] = phrasesDF['text_sentiment'].apply(lambda c: 'best' if c >=0.75 else ('good' if c >=0.5 else ('better' if c >=0.25 else ('ok' if c >=0 else ('bad' if c <=-0.25 else ('very bad' if c <=-0.5 else ('worst' if c <=-0.75 else 'very worst')))))))

In [18]:
phrasesDF.head()

,brand,phrase,rating,aspect,review_id,text_sentiment,text_subjectivity,comp_score
0,samsung,even the note but this screen here bests all t...,4,screen,1,0.000000,0.000000,ok
1,samsung,all this makes for outstanding screen,4,screen,1,0.500000,0.875000,good
2,samsung,Great main camera,4,camera,1,0.483333,0.541667,better
3,samsung,result but all all the camera top notch all its,4,camera,1,0.500000,0.500000,good
4,samsung,find phone with better main camera the market,4,camera,1,0.333333,0.416667,better


In [19]:
phrasesDF['label'] = phrasesDF['comp_score'].apply(lambda c: 4 if c =='best' else (3 if c =='good' else (2 if c =='better' else (1 if c =='ok' else (-1 if c =='bad' else (-2 if c =='very bad' else (-3 if c =='worst' else -4)))))))

In [20]:
phrasesDF.head()

,brand,phrase,rating,aspect,review_id,text_sentiment,text_subjectivity,comp_score,label
0,samsung,even the note but this screen here bests all t...,4,screen,1,0.000000,0.000000,ok,1
1,samsung,all this makes for outstanding screen,4,screen,1,0.500000,0.875000,good,3
2,samsung,Great main camera,4,camera,1,0.483333,0.541667,better,2
3,samsung,result but all all the camera top notch all its,4,camera,1,0.500000,0.500000,good,3
4,samsung,find phone with better main camera the market,4,camera,1,0.333333,0.416667,better,2


In [21]:
def modelEvaluation(predictions, y_test_set):
    #Print model evaluation to predicted result 
    
    print ("\nAccuracy on validation set: {:.4f}".format(accuracy_score(y_test_set, predictions)))
    #print "\nAUC score : {:.4f}".format(roc_auc_score(y_test_set, predictions))
    print ("\nClassification report : \n", metrics.classification_report(y_test_set, predictions))
    print ("\nConfusion Matrix : \n", metrics.confusion_matrix(y_test_set, predictions))

RANDOM FOREST

In [22]:
import future
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn import metrics
from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.preprocessing import label_binarize
from sklearn.ensemble import RandomForestClassifier
from bs4 import BeautifulSoup  
import re
import nltk
from nltk.corpus import stopwords 

In [23]:
#split data into training and testing set
x_train, x_test, y_train, y_test = train_test_split(phrasesDF['phrase'], phrasesDF['label'], test_size=0.3, random_state=0)
    
    
#If the label are words instead of numbers, they can be replaced programmatically using following two lines
#     y_train = label_binarize(y_train, classes=[0, 1, 2])
#     y_test = label_binarize(y_test, classes=[0, 1, 2])

In [24]:
# Preprocess text data in training set and validation set
x_train_cleaned = []
x_test_cleaned = []

for d in x_train:
    x_train_cleaned.append(cleanText(d))

for d in x_test:
    x_test_cleaned.append(cleanText(d)) 

In [27]:
tfidf = TfidfVectorizer(min_df=5) #minimum document frequency of 5
x_train_tfidf = tfidf.fit_transform(x_train)
print ("Number of features : %d \n" %len(tfidf.get_feature_names())) #1722
print ("Show some feature names : \n", tfidf.get_feature_names()[::1000])
 

Number of features : 1890 

Show some feature names : 
 ['00', 'mediocre']


In [28]:
 x_train_input = tfidf.transform(x_train_cleaned)

In [29]:
rand = RandomForestClassifier()
rand.fit(x_train_input, y_train)
y_rand_predicted = rand.predict(tfidf.transform(x_test_cleaned))

In [30]:
modelEvaluation(y_rand_predicted, y_test)


Accuracy on validation set: 0.8881

Classification report : 
               precision    recall  f1-score   support

          -4       0.89      0.82      0.85       594
          -1       0.91      0.84      0.87       473
           1       0.91      0.96      0.93      3147
           2       0.88      0.75      0.81       930
           3       0.84      0.85      0.84       962
           4       0.85      0.92      0.89       534

    accuracy                           0.89      6640
   macro avg       0.88      0.85      0.87      6640
weighted avg       0.89      0.89      0.89      6640


Confusion Matrix : 
 [[ 485   18   77    9    5    0]
 [   9  395   58    1    8    2]
 [  43   12 3011   38   33   10]
 [   8    3  105  698   91   25]
 [   0    4   50   44  816   48]
 [   0    1   13    6   22  492]]


In [34]:
print ("Accuracy of Random Forest = " + str(rand.score(tfidf.transform(x_test_cleaned), y_test)))


Accuracy of Random Forest = 0.8881024096385542


In [32]:
print(rand.predict(tfidf.transform([
    "It was bad"
])))

[-1]
